# Note: this file is old and uses a different format; ignore the messy code

In [1]:
import sys, traceback
print(sys.executable)
print(sys.version)
print(sys.version_info)

import math
import astropy
print('astropy',astropy.__version__ )
from spectral_cube import SpectralCube      # This is a handy package for working with 3D data cubes
from spectral_cube import LazyMask
from astropy.coordinates import SkyCoord
from reproject import reproject_interp      
from reproject.mosaicking import find_optimal_celestial_wcs 
import regions
import reproject
print('reproject',reproject.__version__)
import spectral_cube
print('spectral_cube',spectral_cube.__version__)
import numpy as np                          
import pylab                                
import matplotlib 
import matplotlib.gridspec as gridspec                                                                                             
import scipy
import astropy.io.fits as fits                                                          
from astropy.wcs import WCS                 
from astropy import units as u              
import pyspeckit as psk   
import astrodendro
import matplotlib.colors as colors
from matplotlib import pyplot as plt
# Suppress warnings we don't care about:
import sys
import gc
from astropy.convolution import Gaussian1DKernel
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
%matplotlib inline
from astrodendro.analysis import PPVStatistic
import os

print(astrodendro.__file__)

from matplotlib.patches import Ellipse
import copy
from tqdm import tqdm
import radio_beam
from astropy.table import Table

from scipy.optimize import curve_fit
from scipy.optimize import leastsq

Num_per_kg= 6.0221409*10**23/(2.8*10**-3)#6.0221409*10**23/29.0180*10**-3#num/kg for h2


gal="GC"

dist_cmz = 8.178*10**-3*u.Mpc


#Return a cropped cube for some ra and dec, also crops the velocity axis if needed (0 for no crop)
def Crop(cube,WCS,Np1,Np2,BadVel,D2):
    NraDP1 = [int(WCS.world_to_pixel(Np1)[0]),int(WCS.world_to_pixel(Np1)[1])]
    NraDP2 = [int(WCS.world_to_pixel(Np2)[0]),int(WCS.world_to_pixel(Np2)[1])]
    if(D2==False):
        return cube[BadVel:np.shape(cube)[0]-BadVel,NraDP1[1]:NraDP2[1],NraDP1[0]:NraDP2[0]]
    if(D2==True):
        return cube[NraDP1[1]:NraDP2[1],NraDP1[0]:NraDP2[0]]



def Crop_Nans(data):

    sx,sy,ex,ey=0,0,0,0
    for lmi in range(np.shape(data[0,:,:])[0]):

        if(ey!=0 and sx!=0 and ex!=0 and sy!=0):
            print("F",lmi)
            break
        for lmj in range(np.shape(data[0,:,:])[1]):

            if(sx==0):            
                if(np.nanmean(data[0,lmi,:])>0 or np.nanmean(data[0,lmi,:])<0):
                    sx=lmi


            if(sy==0):
                if(np.nanmean(data[0,:,lmj])>0 or np.nanmean(data[0,:,lmj])<0):
                    sy=lmj

            if(ex==0):
                if(np.nanmean(data[0,np.shape(datn[0,:,:])[0]-lmi-1,:])>0 or np.nanmean(data[0,np.shape(data[0,:,:])[0]-lmi-1,:])<0):
                    ex=np.shape(data[0,:,:])[0]-lmi-1

            if(ey==0):
                if(np.nanmean(data[0,:,np.shape(data[0,:,:])[1]-lmj-1])>0 or np.nanmean(data[0,:,np.shape(data[0,:,:])[1]-lmj-1])<0):
                    ey=np.shape(data[0,:,:])[1]-lmj-1

            if(ey!=0 and ex!=0 and sx!=0 and sy!=0):
                break
    print(sx,ex,sy,ey)
    return sx,ex,sy,ey


from astropy.utils import NumpyRNGContext
def gaussian_beam(f, beam_gauss_width):
    '''
    Fourier transform of a Gaussian beam. NOT the power spectrum (multiply exp
    argument by 2 for power spectrum).
    Parameters
    ----------
    f : np.ndarray
        Frequencies to evaluate beam at.
    beam_gauss_width : float
        Beam size. Should be the Gaussian rms, not FWHM.
    '''
    return np.exp(-f**2 * np.pi**2 * 2 * beam_gauss_width**2)

def gauss_correlated_noise_2D(shape, sigma, beam_gauss_width,
                              randomseed=327485749):
    
    '''
    Generate correlated Gaussian noise with sigma, smoothed by a
    Gaussian kernel.
    '''

    # Making a real signal. Only need real part of FFT
    freqs_yy, freqs_xx = np.meshgrid(np.fft.fftfreq(shape[0]),
                                     np.fft.rfftfreq(shape[1]), indexing="ij")

    freqs = np.sqrt(freqs_yy**2 + freqs_xx**2)
    # freqs[freqs == 0.] = np.NaN
    # freqs[freqs == 0.] = 1.

    imsize = shape[0]

    Np1 = (imsize - 1) // 2 if imsize % 2 != 0 else imsize // 2
    
    with NumpyRNGContext(randomseed):

        angles = np.random.uniform(0, 2 * np.pi,
                                   size=freqs.shape)

    noise = np.cos(angles) + 1j * np.sin(angles)

    if imsize % 2 == 0:
        noise[1:Np1, 0] = np.conj(noise[imsize:Np1:-1, 0])
        noise[1:Np1, -1] = np.conj(noise[imsize:Np1:-1, -1])
        noise[Np1, 0] = noise[Np1, 0].real + 1j * 0.0
        noise[Np1, -1] = noise[Np1, -1].real + 1j * 0.0

    else:
        noise[1:Np1 + 1, 0] = np.conj(noise[imsize:Np1:-1, 0])
        noise[1:Np1 + 1, -1] = np.conj(noise[imsize:Np1:-1, -1])

    # Zero freq components must have no imaginary part to be own conjugate
    noise[0, -1] = noise[0, -1].real + 1j * 0.0
    noise[0, 0] = noise[0, 0].real + 1j * 0.0

    corr_field = np.fft.irfft2(noise *
                               gaussian_beam(freqs, beam_gauss_width))

    norm = (np.sqrt(np.sum(corr_field**2)) / np.sqrt(corr_field.size)) / sigma

    corr_field /= norm
    
    return corr_field
restfreq = 345.79598990 * u.GHz

/home/ben/miniconda3/bin/python
3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
astropy 5.1.dev153+gb740594dc
reproject 0.8
spectral_cube 0.6.1.dev22+g003ef16
/home/ben/.local/lib/python3.8/site-packages/astrodendro/__init__.py


# Fix the headers because they are broken for some reason


In [5]:
#Fix header
files = ["12CO_GC_359-000_mosaic.fit"]
#files=["Pre-Smoothed.fits"]



sc = SpectralCube.read("Spectral Cubes/"+files[0])  
Cube_Name = files[0]+"_Header_Fix.fits"
print(1)
sc.allow_huge_operations=True
sc = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio").to(u.K) # Change units from Hz to km/s
scWsc_copy = scWsc_copy.with_spectral_unit(u.km / u.s, velocity_convention='optical', rest_value=restfreq)

new = SpectralCube(data=sc.hdu.data,wcs =WCS(sc.header),mask=sc.mask)
new.allow_huge_operations=True
new=new*sc[0][0][0].unit
print(2)
del sc
sc=new
print(3)
del new

sc.write("Spectral Cubes/"+Cube_Name,overwrite=True)
print(4)
print("done")


1
2
3
4
done


In [12]:
#Just Using CO 3-2 right now
#Fix the header
files = ["12CO_GC_001-002_mosaic.fit"]
#files=["Pre-Smoothed.fits"]



sc = SpectralCube.read("Spectral Cubes/"+files[0])  
Cube_Name = files[0]+"_Header_Fix.fits"
print(1)
sc.allow_huge_operations=True
sc = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio").to(u.K) # Change units from Hz to km/s
new = SpectralCube(data=sc.hdu.data,wcs =WCS(sc.header),mask=sc.mask)
new.allow_huge_operations=True
new=new*sc[0][0][0].unit
print(2)
del sc
sc=new
print(3)
del new

sc.write("Spectral Cubes/"+Cube_Name,overwrite=True)
print(4)
print("done")

1
2
3
4
done


In [7]:
#Just Using CO 3-2 right now
#Fix the header
files = ["12CO_GC_357-358_mosaic.fit"]
#files=["Pre-Smoothed.fits"]



sc = SpectralCube.read("Spectral Cubes/"+files[0])  
Cube_Name = files[0]+"_Header_Fix.fits"
print(1)
sc.allow_huge_operations=True
sc = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio").to(u.K) # Change units from Hz to km/s
new = SpectralCube(data=sc.hdu.data,wcs =WCS(sc.header),mask=sc.mask)
new.allow_huge_operations=True
new=new*sc[0][0][0].unit
print(2)
del sc
sc=new
print(3)
del new

sc.write("Spectral Cubes/"+Cube_Name,overwrite=True)
print(4)
print("done")

1
2
3
4
done


In [8]:
#Just Using CO 3-2 right now
#Fix the header
files = ["12CO_GC_003-005_mosaic.fit"]
#files=["Pre-Smoothed.fits"]



sc = SpectralCube.read("Spectral Cubes/"+files[0])  
Cube_Name = files[0]+"_Header_Fix.fits"
print(1)
sc.allow_huge_operations=True
sc = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio").to(u.K) # Change units from Hz to km/s
new = SpectralCube(data=sc.hdu.data,wcs =WCS(sc.header),mask=sc.mask)
new.allow_huge_operations=True
new=new*sc[0][0][0].unit
print(2)
del sc
sc=new
print(3)
del new

sc.write("Spectral Cubes/"+Cube_Name,overwrite=True)
print(4)
print("done")

1
2
3
4
done


# Assemble the different data cubes to get the full FOV around the center

In [23]:
# This process is apparently similar to the casa task imregrid and immath, which just combines images together along the same pixel grid.
# Where there is overlapping data, I take the mean.
# It is difficult to say if this is the only approach, since the overlapping data sometimes has notable different intensity values.
# Perhaps reducing the raw data (from CANFAR) will provide a better result at these overlap points.

In [1]:
%run ./"0 Imports and functions.ipynb"

/home/ben/miniconda3/bin/python
3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
astropy 5.1.dev153+gb740594dc
spectral_cube 0.6.1.dev22+g003ef16
spectral_cube file path /home/ben/.local/lib/python3.8/site-packages/spectral_cube/__init__.py
reproject 0.8
1.23.1 Numpy
astrodendro_file: /home/ben/.local/lib/python3.8/site-packages/astrodendro/__init__.py
Results will be saved to Directory ./Result Files
Cubes will be saved and loaded with Directory ./Spectral Cubes
Plots will be saved to Directory ./Plots
Dendrograms will be saved to Directory ./Dendrograms


# middle-right cube

# reproject cubes to a common grid

In [2]:
File_Name = "12CO_GC_359-000_mosaic.fit"+"_Header_Fix.fits" #the name of the initial SC file.
scB = SpectralCube.read("Spectral Cubes/"+File_Name) 
for i in range(10):
    
    print('begin step:',i,"of",int(len(scB)/60) +1)

    Cube_Name_Save = str(i)+"_"+File_Name+"_Reproject"+'.fits'

    n = scB[i*60:i*60+60]
    n.write("h_interim.fits",overwrite=True)
    #sc = copy.deepcopy(scB[i*i_step:i*i_step+i_step])
    sc = SpectralCube.read("h_interim.fits") #need to do this because python has glitches with pointers and copying
            
    scB.allow_huge_operations=True
    sc.allow_huge_operations=True
    
    reheader = sc.header

    
    if reheader['cdelt1']>0:
        origin_x = 357.1
    else:
        origin_x = 2.9

    if reheader['cdelt2']>0:
        origin_y = -0.5#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value

    else:
        origin_y = 0.5#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value


    npix_x = abs(int(5.8/reheader['cdelt1']))
    npix_y = abs(int(1/reheader['cdelt2']))

    reheader['naxis1'] = npix_x
    reheader['naxis2'] = npix_y
    print(npix_x,npix_y,reheader['cdelt1'])
    reheader['crval1'] = origin_x
    reheader['crval2'] = origin_y

    reheader['crpix1'] = 0
    reheader['crpix2'] = 0

    reheader['CTYPE1'] = "GLON-TAN"
    reheader['CTYPE2'] = "GLAT-TAN"
    del reheader['lonpole']
    del reheader['latpole']
    del reheader['wcsaxes']#Dont need these anymore, 




    print('start reprojection\n')
    print('check max SC value:',np.nanmax(sc),"SC shape:", np.shape(sc))#These should be a non zero float and the shape of the cube (30,~1000,~1000)

    sc.allow_huge_operations=True
    sc2 = sc.reproject(reheader, order='bilinear', use_memmap=True, filled=True) # Had to change reproject.py so it deletes output.np before making a new one


    del sc # save space

    # make a new cube with the reprojcted data (remove all the logs from the old cube)

    new = SpectralCube(data=sc2.hdu.data,wcs =WCS(sc2.header),header=sc2.header,mask=sc2.mask)
    new.allow_huge_operations=True
    new = new*sc2[0][0][0].unit

    #do this because scs dont like being modified
    del sc2
    sc2 = new
    del new

    print('\nend reprojection\n')
    print('check max SC value:',np.nanmax(sc2),"SC shape:", np.shape(sc2))#These should ALSO be a non zero float and the shape of the cube (30,~1000,~1000)

    sc = sc2
    del sc2
    sc.allow_huge_operations=True

    sc.write("Spectral Cubes/"+Cube_Name_Save,overwrite=True)

    del sc

   


begin step: 0 of 13
3479 599 -0.0016666666666693
start reprojection

check max SC value: 16.86088 SC shape: (60, 650, 1250)

end reprojection

check max SC value: 11.006666 SC shape: (60, 599, 3479)
begin step: 1 of 13
3479 599 -0.0016666666666693
start reprojection

check max SC value: 12.833167 SC shape: (60, 650, 1250)

end reprojection

check max SC value: 8.576083 SC shape: (60, 599, 3479)
begin step: 2 of 13
3479 599 -0.0016666666666693
start reprojection

check max SC value: 10.988167 SC shape: (60, 650, 1250)

end reprojection

check max SC value: 10.029535 SC shape: (60, 599, 3479)
begin step: 3 of 13
3479 599 -0.0016666666666693
start reprojection

check max SC value: 21.37605 SC shape: (60, 650, 1250)

end reprojection

check max SC value: 21.042341 SC shape: (60, 599, 3479)
begin step: 4 of 13
3479 599 -0.0016666666666693
start reprojection

check max SC value: 39.25378 SC shape: (60, 650, 1250)

end reprojection

check max SC value: 38.757698 SC shape: (60, 599, 3479)
begi

# Splice the piecewise (velocity) cubes

In [ ]:
File_Name = "12CO_GC_359-000_mosaic.fit"+"_Header_Fix.fits" #the name of the initial SC file.
    
Splice_vels(Cube_Name_Load=File_Name+"_Reproject"+'.fits',istart=1,iend=4,Save="1_I_Quall_RH.fits")

Splice_vels(Cube_Name_Load=File_Name+"_Reproject"+'.fits',istart=4,iend=7,Save="2_I_Quall_RH.fits")
gc.collect()
Splice_vels(Cube_Name_Load=File_Name+"_Reproject"+'.fits',istart=7,iend=9,Save="3_I_Quall_RH.fits")

Loaded 1_12CO_GC_359-000_mosaic.fit_Header_Fix.fits_Reproject.fits
Loaded 2_12CO_GC_359-000_mosaic.fit_Header_Fix.fits_Reproject.fits
Loaded 3_12CO_GC_359-000_mosaic.fit_Header_Fix.fits_Reproject.fits
Loaded 4_12CO_GC_359-000_mosaic.fit_Header_Fix.fits_Reproject.fits
Loaded 5_12CO_GC_359-000_mosaic.fit_Header_Fix.fits_Reproject.fits
Loaded 6_12CO_GC_359-000_mosaic.fit_Header_Fix.fits_Reproject.fits


In [6]:
File_Name = "12CO_GC_359-000_mosaic.fit"+"_Header_Fix.fits"+"_Reproject.fits" #the name of the initial SC file.

# Splice the partwise cubes back together:
scs=[]
for i in range(99):
    try: 
        Name_p = str(i)+"_"+File_Name
        sc=SpectralCube.read(("Spectral Cubes/"+Name_p))
        scs.append(sc)
    except:
        print(Name_p)
        break

reheader = scs[0].header
rewcs = scs[0].wcs
mask = scs[0].mask.include() # Need to create a mask because it doesn't get splcied
print(np.shape(mask),np.shape(scs[0][:].hdu.data))
print("1")
scW = np.concatenate((scs[0][:].hdu.data,scs[1][:].hdu.data,scs[2][:].hdu.data,scs[3][:].hdu.data,scs[4][:].hdu.data,scs[5][:].hdu.data,scs[6][:].hdu.data,scs[7][:].hdu.data,scs[8][:].hdu.data),dtype = type(sc))
print("1")
mask = np.concatenate((mask[:],scs[1][:].mask.include(),scs[2][:].mask.include(),scs[3][:].mask.include(),scs[4][:].mask.include(),scs[5][:].mask.include(),scs[6][:].mask.include(),scs[7][:].mask.include(),scs[8][:].mask.include()),dtype = type(sc[:].mask.include()))
print("1")
scUNIT=copy.deepcopy(scs[0][0][0][0].unit)

# This only matters for formatting reasons:
def duh(lol):
    gp = np.where(lol!=np.nan)
    lol[gp]=True
    return lol # Anywhere that has data will be unmasked

reheader["NAXIS3"] = len(scW)
Full_Mask = LazyMask(function = duh,data = mask, wcs = rewcs)

print(np.shape(mask),np.shape(scW))
scWsc = SpectralCube(data = scW,wcs = rewcs, header = reheader, mask = Full_Mask)# The spliced cube

scWsc.allow_huge_operations=True
scWsc = scWsc*scUNIT#Add unit back in


scWsc.write("Spectral Cubes/"+"Looooooooooo.fits",overwrite=True)

    
del scWsc
del Full_Mask
del reheader
del scW
del rewcs
gc.collect()



10_12CO_GC_359-000_mosaic.fit_Header_Fix.fits_Reproject.fits
(3, 599, 3479) (3, 599, 3479)
1
1
1
(27, 599, 3479) (27, 599, 3479)


771

# Use memmap to splice the reprojected cubes

In [14]:
def delete_memmap_file(file_path):
    if os.path.exists(file_path):
        try:
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        except PermissionError:
            print(f"File in use, unable to delete: {file_path}")

In [10]:
import numpy as np
import gc
import copy
from spectral_cube import SpectralCube, LazyMask

File_Name = "12CO_GC_359-000_mosaic.fit"+"_Header_Fix.fits"+"_Reproject.fits" # Initial file name

# Load spectral cubes part-wise
scs = []
for i in range(99):
    try:
        Name_p = str(i) + "_" + File_Name
        sc = SpectralCube.read("Spectral Cubes/" + Name_p)
        scs.append(sc)
    except:
        print(f"File missing: {Name_p}")
        break

# Extract the header, WCS, and mask from the first cube
reheader = scs[0].header
rewcs = scs[0].wcs
mask = scs[0].mask.include()  # Extract the mask separately

# Get the shape of the full concatenated cube
cube_shape = (sum(sc.shape[0] for sc in scs),) + scs[0].shape[1:]  # (Total channels, X, Y)

# Create a memory-mapped file for the concatenated cube
memmap_file = "Spectral Cubes/concatenated_cube.dat"
delete_memmap_file(memmap_file) #try to delete it if it exists
scW = np.memmap(memmap_file, dtype=scs[0].hdu.data.dtype, mode='w+', shape=cube_shape)

# Fill the memmap file sequentially
start_idx = 0
for sc in scs:
    end_idx = start_idx + sc.shape[0]
    scW[start_idx:end_idx] = sc[:].hdu.data
    start_idx = end_idx
scW.flush()
gc.collect()

# Create a memory-mapped file for the mask
mask_memmap_file = "Spectral Cubes/mask.dat"
mask_memmap = np.memmap(mask_memmap_file, dtype=bool, mode='w+', shape=cube_shape)

# Fill the mask memmap
start_idx = 0
for sc in scs:
    end_idx = start_idx + sc.shape[0]
    mask_memmap[start_idx:end_idx] = sc.mask.include()
    start_idx = end_idx
mask_memmap.flush()
gc.collect()

# Retrieve the unit
scUNIT = copy.deepcopy(scs[0][0][0][0].unit)

# Mask function
def duh(lol):
    gp = np.where(lol != np.nan)
    lol[gp] = True
    return lol  # Unmask data

# Update header
reheader["NAXIS3"] = cube_shape[0]
Full_Mask = LazyMask(function=duh, data=mask_memmap, wcs=rewcs)

print("Final cube shape:", scW.shape)
scWsc = SpectralCube(data=scW, wcs=rewcs, header=reheader, mask=Full_Mask)  # Create SpectralCube

scWsc.allow_huge_operations = True
scWsc = scWsc * scUNIT  # Restore unit

# Write to FITS file
scWsc.write("Spectral Cubes/12CO_GC_359-000_Reprojected.fits", overwrite=True)
# Cleanup
del scWsc
del Full_Mask
del reheader
del scW
del rewcs
gc.collect()


File missing: 10_12CO_GC_359-000_mosaic.fit_Header_Fix.fits_Reproject.fits
Final cube shape: (600, 599, 3479)


36

In [15]:
import numpy as np
import gc
import copy
from spectral_cube import SpectralCube, LazyMask

File_Name = "12CO_GC_357-358_mosaic.fit"+"_Header_Fix.fits"+"_Reproject.fits" # Initial file name

# Load spectral cubes part-wise
scs = []
for i in range(99):
    try:
        Name_p = str(i) + "_" + File_Name
        sc = SpectralCube.read("Spectral Cubes/" + Name_p)[0:3][0:5]
        scs.append(sc)
    except:
        print(f"File missing: {Name_p}")
        break

# Extract the header, WCS, and mask from the first cube
reheader = scs[0].header
rewcs = scs[0].wcs
mask = scs[0].mask.include()  # Extract the mask separately

# Get the shape of the full concatenated cube
cube_shape = (sum(sc.shape[0] for sc in scs),) + scs[0].shape[1:]  # (Total channels, X, Y)

# Create a memory-mapped file for the concatenated cube
memmap_file = "Spectral Cubes/concatenated_cube.dat"
delete_memmap_file(memmap_file) #try to delete it if it exists
scW = np.memmap(memmap_file, dtype=scs[0].hdu.data.dtype, mode='w+', shape=cube_shape)

# Fill the memmap file sequentially
start_idx = 0
for sc in scs:
    end_idx = start_idx + sc.shape[0]
    print(sc.hdu.data)
    scW[start_idx:end_idx] = sc.hdu.data
    start_idx = end_idx
scW.flush()

# Create a memory-mapped file for the mask
mask_memmap_file = "Spectral Cubes/mask.dat"
delete_memmap_file(mask_memmap_file) #try to delete it if it exists
mask_memmap = np.memmap(mask_memmap_file, dtype=bool, mode='w+', shape=cube_shape)

# Fill the mask memmap
start_idx = 0
for sc in scs:
    end_idx = start_idx + sc.shape[0]
    mask_memmap[start_idx:end_idx] = sc.mask.include()
    start_idx = end_idx
mask_memmap.flush()

# Retrieve the unit
scUNIT = copy.deepcopy(scs[0][0][0][0].unit)

# Mask function
def duh(lol):
    gp = np.where(lol != np.nan)
    lol[gp] = True
    return lol  # Unmask data

# Update header
reheader["NAXIS3"] = cube_shape[0]
Full_Mask = LazyMask(function=duh, data=mask_memmap, wcs=rewcs)

print("Final cube shape:", scW.shape)
scWsc = SpectralCube(data=scW, wcs=rewcs, header=reheader, mask=Full_Mask)  # Create SpectralCube

scWsc.allow_huge_operations = True
scWsc = scWsc * scUNIT  # Restore unit

# Write to FITS file
scWsc.write("Spectral Cubes/12CO_GC_357-358_Reprojected.fits", overwrite=True)
# Cleanup
del scWsc
del Full_Mask
del reheader
del scW
del rewcs
gc.collect()


File missing: 10_12CO_GC_357-358_mosaic.fit_Header_Fix.fits_Reproject.fits
Deleted: Spectral Cubes/concatenated_cube.dat
[[[        nan         nan         nan ...  0.01071595  0.9030936
    0.3550301 ]
  [        nan         nan         nan ...  0.71415067  1.7458211
    1.0098869 ]
  [        nan         nan         nan ...  0.56579494  1.3485703
    1.4092424 ]
  ...
  [        nan         nan         nan ... -0.61089796 -0.18373927
    0.21237355]
  [        nan         nan         nan ... -0.03583494 -0.02605736
    0.22870214]
  [        nan         nan         nan ... -0.2635784  -0.5660332
    0.1452115 ]]

 [[        nan         nan         nan ... -0.03603706  0.42515957
    0.15591466]
  [        nan         nan         nan ...  0.39616674  0.8638516
    0.09241658]
  [        nan         nan         nan ...  0.25327694  0.6026627
    0.4409315 ]
  ...
  [        nan         nan         nan ... -0.42398134 -0.18207595
    0.81640726]
  [        nan         nan         nan ..

108

In [18]:
import numpy as np
import gc
import copy
from spectral_cube import SpectralCube, LazyMask

File_Name = "12CO_GC_001-002_mosaic.fit"+"_Header_Fix.fits"+"_Reproject.fits" # Initial file name


# Load spectral cubes part-wise
scs = []
for i in range(99):
    try:
        Name_p = str(i) + "_" + File_Name
        sc = SpectralCube.read("Spectral Cubes/" + Name_p)[:,0:5,:]
        scs.append(sc)
    except:
        print(f"File missing: {Name_p}")
        break

# Extract the header, WCS, and mask from the first cube
reheader = scs[0].header
rewcs = scs[0].wcs
mask = scs[0].mask.include()  # Extract the mask separately

# Get the shape of the full concatenated cube
cube_shape = (sum(sc.shape[0] for sc in scs),) + scs[0].shape[1:]  # (Total channels, X, Y)

# Create a memory-mapped file for the concatenated cube
memmap_file = "Spectral Cubes/concatenated_cube.dat"
delete_memmap_file(memmap_file) #try to delete it if it exists
scW = np.memmap(memmap_file, dtype=scs[0].hdu.data.dtype, mode='w+', shape=cube_shape)

# Fill the memmap file sequentially
start_idx = 0
for sc in scs:
    end_idx = start_idx + sc.shape[0]
    scW[start_idx:end_idx] = sc.hdu.data
    start_idx = end_idx
scW.flush()


# Create a memory-mapped file for the mask
mask_memmap_file = "Spectral Cubes/mask.dat"
mask_memmap = np.memmap(mask_memmap_file, dtype=bool, mode='w+', shape=cube_shape)

# Fill the mask memmap
start_idx = 0
for sc in scs:
    end_idx = start_idx + sc.shape[0]
    mask_memmap[start_idx:end_idx] = sc.mask.include()
    start_idx = end_idx
mask_memmap.flush()

# Retrieve the unit
scUNIT = copy.deepcopy(scs[0][0][0][0].unit)

# Mask function
def duh(lol):
    gp = np.where(lol != np.nan)
    lol[gp] = True
    return lol  # Unmask data

# Update header
reheader["NAXIS3"] = cube_shape[0]
Full_Mask = LazyMask(function=duh, data=mask_memmap, wcs=rewcs)

print("Final cube shape:", scW.shape)
scWsc = SpectralCube(data=scW, wcs=rewcs, header=reheader, mask=Full_Mask)  # Create SpectralCube

scWsc.allow_huge_operations = True
scWsc = scWsc * scUNIT  # Restore unit

# Write to FITS file
scWsc.write("Spectral Cubes/12CO_GC_001-002_Reprojected.fits", overwrite=True)

# Cleanup
del scWsc
del Full_Mask
del reheader
del scW
del rewcs
gc.collect()

File missing: 10_12CO_GC_001-002_mosaic.fit_Header_Fix.fits_Reproject.fits
Deleted: Spectral Cubes/concatenated_cube.dat
[[[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]

 ...

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]

88

In [ ]:
Splice_vels(Cube_Name_Load='I_Quall_RH.fits')

In [ ]:
Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

# Load the spliced cube

sc = SpectralCube.read(("Spectral Cubes/"+Cube_Name_Save)) 

# Reproject the spliced Cube's veloctiy axis

Repo_Velocity(sc,Cube_Name_Save,Cube_Information)

# Here is the new cube's name

Cube_Name_Save = Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

del sc

gc.collect()

# middle-left cube

In [26]:
File_Name = "12CO_GC_001-002_mosaic.fit"+"_Header_Fix.fits" #the name of the initial SC file.
scB = SpectralCube.read("Spectral Cubes/"+File_Name) 
istep=2
for i in range(18,22):
    
    print('\nbegin step:',i,"of",int(len(scB)/istep) +1)

    Cube_Name_Save = str(i)+"_"+File_Name+"_Reproject"+'.fits'

    n = scB[i*istep:i*istep+istep,0:5,0:5]
    n.write("h_interim.fits",overwrite=True)
    #sc = copy.deepcopy(scB[i*i_step:i*i_step+i_step])
    sc = SpectralCube.read("h_interim.fits") #need to do this because python has glitches with pointers and copying
            
    scB.allow_huge_operations=True
    sc.allow_huge_operations=True
    
    File_Name_R = "12CO_GC_359-000_mosaic.fit"+"_Header_Fix.fits" #the name of the initial SC file.
    reheader = SpectralCube.read("Spectral Cubes/"+File_Name_R)[i*istep:i*istep+istep,0:5,0:5].header

    

    
    if reheader['cdelt1']>0:
        origin_x = 357.1
    else:
        origin_x = 2.9

    if reheader['cdelt2']>0:
        origin_y = -0.5#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value

    else:
        origin_y = 0.5#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value


    npix_x = abs(int(5.8/reheader['cdelt1']))
    npix_y = abs(int(1/reheader['cdelt2']))

    reheader['naxis1'] = npix_x
    reheader['naxis2'] = npix_y
    print(npix_x,npix_y,reheader['cdelt1'])

    reheader['crval1'] = origin_x
    reheader['crval2'] = origin_y

    reheader['crpix1'] = 0
    reheader['crpix2'] = 0

    reheader['CTYPE1'] = "GLON-TAN"
    reheader['CTYPE2'] = "GLAT-TAN"
    del reheader['lonpole']
    del reheader['latpole']
    del reheader['wcsaxes']#Dont need these anymore, 



    print(reheader['cdelt1'])
    print('start reprojection\n')
    print('check max SC value:',np.nanmax(sc),"SC shape:", np.shape(sc))#These should be a non zero float and the shape of the cube (30,~1000,~1000)

    sc.allow_huge_operations=True
    print(sc.header)
    print()
    print(reheader)
    sc2 = sc.reproject(reheader, order='bilinear', use_memmap=True, filled=True) # Had to change reproject.py so it deletes output.np before making a new one


    del sc # save space

    # make a new cube with the reprojcted data (remove all the logs from the old cube)

    new = SpectralCube(data=sc2.hdu.data,wcs =WCS(sc2.header),header=sc2.header,mask=sc2.mask)
    new.allow_huge_operations=True
    new = new*sc2[0][0][0].unit

    #do this because scs dont like being modified
    del sc2
    sc2 = new
    del new

    print('\nend reprojection\n')
    print('check max SC value:',np.nanmax(sc2),"SC shape:", np.shape(sc2))#These should ALSO be a non zero float and the shape of the cube (30,~1000,~1000)

    sc = sc2
    del sc2
    sc.allow_huge_operations=True
    print(sc.header['cdelt1'])
    sc.write("Spectral Cubes/"+Cube_Name_Save,overwrite=True)

    del sc

   



begin step: 18 of 392
3479 599 -0.0016666666666693
-0.0016666666666693
start reprojection

check max SC value: nan SC shape: (2, 5, 5)
SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -32 / array data type                                NAXIS   =                    3                                                  NAXIS1  =                    5                                                  NAXIS2  =                    5                                                  NAXIS3  =                    2                                                  BUNIT   = 'K       '                                                            SLICE   = '[[(36, 38, None), (0, 5, None), (0, 5, None)]]'                      WCSAXES =                    3 / Number of coordinate axes                      CRPIX1  =             1125.933 / Pixel coordinate of reference point            CRPIX2  =              125.843 / Pixel coordinate of reference p

KeyboardInterrupt: 

# Splice the piecewise (velocity) cubes

In [ ]:
File_Name = "12CO_GC_001-002_mosaic.fit"+"_Header_Fix.fits" #the name of the initial SC file.
Cube_Name_Save = File_Name+"_Reproject"+'.fits'
    
Splice_vels(Cube_Name_Load=File_Name+"_Reproject"+'.fits',istart=1,iend=4,Save="1_I_Quall_LH.fits")

Splice_vels(Cube_Name_Load=File_Name+"_Reproject"+'.fits',istart=4,iend=7,Save="2_I_Quall_LH.fits")
gc.collect()
Splice_vels(Cube_Name_Load=File_Name+"_Reproject"+'.fits',istart=7,iend=9,Save="3_I_Quall_LH.fits")
print("done")

In [ ]:
Splice_vels(Cube_Name_Load='I_Quall_LH.fits')

In [ ]:
Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

# Load the spliced cube

sc = SpectralCube.read(("Spectral Cubes/"+Cube_Name_Save)) 

# Reproject the spliced Cube's veloctiy axis

Repo_Velocity(sc,Cube_Name_Save,Cube_Information)

# Here is the new cube's name

Cube_Name_Save = Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

del sc

gc.collect()

# The right-most part of the cube

In [7]:
File_Name = "12CO_GC_357-358_mosaic.fit"+"_Header_Fix.fits" #the name of the initial SC file.
scB = SpectralCube.read("Spectral Cubes/"+File_Name) 
istep=60
for i in range(10):
    
    print('begin step:',i,"of",int(len(scB)/istep) +1)

    Cube_Name_Save = str(i)+"_"+File_Name+"_Reproject"+'.fits'

    n = scB[i*istep:i*istep+istep]
    n.write("h_interim.fits",overwrite=True)
    #sc = copy.deepcopy(scB[i*i_step:i*i_step+i_step])
    sc = SpectralCube.read("h_interim.fits") #need to do this because python has glitches with pointers and copying
            
    scB.allow_huge_operations=True
    sc.allow_huge_operations=True
    
    File_Name_R = "12CO_GC_359-000_mosaic.fit"+"_Header_Fix.fits" #the name of the initial SC file.
    reheader = SpectralCube.read("Spectral Cubes/"+File_Name_R)[i*istep:i*istep+istep].header
    
    if reheader['cdelt1']>0:
        origin_x = 357.1
    else:
        origin_x = 2.9

    if reheader['cdelt2']>0:
        origin_y = -0.5#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value

    else:
        origin_y = 0.5#sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value


    npix_x = abs(int(5.8/reheader['cdelt1']))
    npix_y = abs(int(1/reheader['cdelt2']))

    reheader['naxis1'] = npix_x
    reheader['naxis2'] = npix_y
    print(npix_x,npix_y,reheader['cdelt1'])
    reheader['crval1'] = origin_x
    reheader['crval2'] = origin_y

    reheader['crpix1'] = 0
    reheader['crpix2'] = 0

    reheader['CTYPE1'] = "GLON-TAN"
    reheader['CTYPE2'] = "GLAT-TAN"
    del reheader['lonpole']
    del reheader['latpole']
    del reheader['wcsaxes']#Dont need these anymore, 




    print('start reprojection\n')
    print('check max SC value:',np.nanmax(sc),"SC shape:", np.shape(sc))#These should be a non zero float and the shape of the cube (30,~1000,~1000)

    sc.allow_huge_operations=True
    sc2 = sc.reproject(reheader, order='bilinear', use_memmap=True, filled=True) # Had to change reproject.py so it deletes output.np before making a new one


    del sc # save space

    # make a new cube with the reprojcted data (remove all the logs from the old cube)

    new = SpectralCube(data=sc2.hdu.data,wcs =WCS(sc2.header),header=sc2.header,mask=sc2.mask)
    new.allow_huge_operations=True
    new = new*sc2[0][0][0].unit

    #do this because scs dont like being modified
    del sc2
    sc2 = new
    del new

    print('\nend reprojection\n')
    print('check max SC value:',np.nanmax(sc2),"SC shape:", np.shape(sc2))#These should ALSO be a non zero float and the shape of the cube (30,~1000,~1000)

    sc = sc2
    del sc2
    sc.allow_huge_operations=True

    sc.write("Spectral Cubes/"+Cube_Name_Save,overwrite=True)

    del sc

   


begin step: 0 of 14
3479 599 -0.0016666666666693
start reprojection

check max SC value: 20.751663 SC shape: (60, 651, 1250)

end reprojection

check max SC value: 13.248677 SC shape: (60, 599, 3479)
begin step: 1 of 14
3479 599 -0.0016666666666693
start reprojection

check max SC value: 22.16095 SC shape: (60, 651, 1250)

end reprojection

check max SC value: 12.328638 SC shape: (60, 599, 3479)
begin step: 2 of 14
3479 599 -0.0016666666666693
start reprojection

check max SC value: 16.637972 SC shape: (60, 651, 1250)

end reprojection

check max SC value: 10.630054 SC shape: (60, 599, 3479)
begin step: 3 of 14
3479 599 -0.0016666666666693
start reprojection

check max SC value: 16.43262 SC shape: (60, 651, 1250)

end reprojection

check max SC value: 9.784449 SC shape: (60, 599, 3479)
begin step: 4 of 14
3479 599 -0.0016666666666693
start reprojection

check max SC value: 16.93418 SC shape: (60, 651, 1250)

end reprojection

check max SC value: 10.905957 SC shape: (60, 599, 3479)
begi

# Splice the piecewise (velocity) cubes

In [ ]:
File_Name = "12CO_GC_357-358_mosaic.fit"+"_Header_Fix.fits" #the name of the initial SC file.
Cube_Name_Save = File_Name+"_Reproject"+'.fits'
    

Splice_vels(Cube_Name_Load=File_Name+"_Reproject"+'.fits',istart=1,iend=4,Save="1_I_Quall_FRH.fits")

Splice_vels(Cube_Name_Load=File_Name+"_Reproject"+'.fits',istart=4,iend=7,Save="2_I_Quall_FRH.fits")
gc.collect()
Splice_vels(Cube_Name_Load=File_Name+"_Reproject"+'.fits',istart=7,iend=9,Save="3_I_Quall_FRH.fits")
print("Done")

In [ ]:
Splice_vels(Cube_Name_Load='I_Quall_FRH.fits')

In [ ]:
Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

# Load the spliced cube

sc = SpectralCube.read(("Spectral Cubes/"+Cube_Name_Save)) 

# Reproject the spliced Cube's veloctiy axis

Repo_Velocity(sc,Cube_Name_Save,Cube_Information)

# Here is the new cube's name

Cube_Name_Save = Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

del sc

gc.collect()

# The left-most part of the cube

In [13]:
# Pointing_Information

Pointing_Information = {}

# This is the middle-right of the GC
Side="FLH_"

#This is the stuff that needs changing between cubes

Pointing_Information["Original_File_Name"] = "12CO_GC_003-005_mosaic.fit"+"_Header_Fix.fits" #the name of the initial SC file.

File_Descriptor='_cmz_CO_J3_2_Splice_'+Side
Pointing_Information["File_Descriptor"] = File_Descriptor
Pointing_Information["target"] = "CMZ"#
Pointing_Information["center"] = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic') #the center of the cmz
Pointing_Information["crop_center"] = SkyCoord('00d00m00.00s  ', '-00d00m00.000s', frame='galactic') #the approximate center of the image, used for cropping

desired_beam_size = 3*u.pc #I add this to the PI later. choose this based on the largest common beam size between the compared observations
distance = 8.178*10**3*u.pc
Pointing_Information["distance"] = distance.to(u.Mpc) #in Mpc
Pointing_Information["target_image_rotation"]=0*u.deg #this is the rotation of the specific image, not the target. (use clockwise rotation angle)
Pointing_Information["target_inclination"]=0*u.deg
Pointing_Information["target_velocity"]=0*u.km/u.s #the speed NGC253 is moving away from us
Pointing_Information["vaxis"]=0 #which axis is the velocity
Pointing_Information["desired_velocity_resolution"] = 2.5*u.km/u.s
ovs = 5 #how much do you desire to oversample the beam by
desired_FOV = [140,700]*u.pc# pc normal to the disk, across the disk
Pointing_Information["Corresponding_Continuum"] = 'gc_850micron_dust.fits' # this is for the band 7, not for HCN J1-0 #The continuum image for this band
#                                                 '4.3pc_beam_CMZ_850um_Cont_140x800pc.fits' # this is the JCMT continuum for 850microns (the CO 3-2 and HCN, HCO 4-3)


print("done")

gc.collect




done


<function gc.collect(generation=2)>

In [14]:

#This stuff will input automagically if the rest is correct

sc = SpectralCube.read("Spectral Cubes/"+Pointing_Information["Original_File_Name"])
header = sc.header
print()
try:
    freq = header["RESTFREQ"]*u.Hz# assuming the header is in Hz
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFREQ"]#
    Pointing_Information['restfreq']=freq         
except:
    freq = header["RESTFRQ"]*u.Hz#
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFRQ"]#            
    Pointing_Information['restfreq']=freq 

######calculate teh beam size of the original image:

if (header['CUNIT1'].find("deg")!=-1):
    CUNIT = 1*u.degree
    Pointing_Information["CUNIT"]=CUNIT
else:
    print("The header should show CUNIT in degrees. If not, just fix this or write CUNIT = the unit it says in the header")
    del jhgasdhgjkahsdkgdfjhsgjgjsdhkfjghjd #this causes an error and stops execution 

    
Pointing_Information["desired_beam_size"] = desired_beam_size #ill put it in a circular beam at this size

Pointing_Information['desired_beam_oversampling'] = ovs
#Pointing_Information['orig_FOV']= Crop_Around_Center(sc,Pointing_Information['target_image_rotation'],Pointing_Information['center'],desired_FOV,Pointing_Information['distance'])[1] #returns the current fov
Pointing_Information['desired_FOV']=desired_FOV

######

Cube_Information = Update_Cube_Information(Pointing_Information,Pointing_Information["Original_File_Name"])






In [22]:
#

    
#Reproject using the function
#sc =  Reproject_To_Region(File=files[0],Prime_Beam=Prime_Beam,Gal="GC",ovs=ovs,FOV=FOVp,Force_Origins=Force_Origins,Force_Value_x=Force_Value_x,Force_Value_y=Force_Value_y)
gc.collect()

#

Reproject_To_Region(Pointing_Information,Cube_Information,i_step=99,Cube_Name_Save='',Force_Origin=[True,[357,3],[-.6,.6]],crop_nans=False)
    
# This is the cube name theyre saved to unless its changed inside the reproject function

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

gc.collect()

print("done")

    

begin step: 0 of 8
begin step: 1 of 8
start beam convolution
The initial image has no beam because it's not interferometric, so one will be created using the pixel size as the initial beam size
0.0 0.0
0.0004389895181039466 0.00043898951810396493
convolve end

fov crop start 

Center: <SkyCoord (Galactic): (l, b) in deg
    (0., -0.)> Pixel center: 3127 324 Pixel FOV: [294, 1471]
cropping cube. rotation: 0.0 deg center: <SkyCoord (Galactic): (l, b) in deg
    (0., -0.)> crop to: [140. 700.] pc
Cropped to  [140. 700.] pc
zero-size array to reduction operation maximum which has no identity
Failed (unless this says attempt to get argmin of empty sequence)
------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_261389/3598772191.py", line 151, in Reproject_To_Region
    print('check max SC value:',np.nanmax(sc),"SC shape:", np.shape(sc))#These should be a non zero float and the shape of the cube (30,~1000,~1000)
  File "<__arra

KeyboardInterrupt: 

# Splice the piecewise (velocity) cubes

In [ ]:
# First, splice the partwise cubes back together

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Splice_vels(Cube_Name_Load=Cube_Name_Save,istart=1)

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

gc.collect()

print("done")

In [ ]:
Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

# Load the spliced cube

sc = SpectralCube.read(("Spectral Cubes/"+Cube_Name_Save)) 

# Reproject the spliced Cube's veloctiy axis

Repo_Velocity(sc,Cube_Name_Save,Cube_Information)

# Here is the new cube's name

Cube_Name_Save = Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

del sc

gc.collect()

# Splicing (spacewise)

In [11]:
def Find_Edges(data):

    sx,sy,ex,ey=0,0,0,0
    for lmi in range(np.shape(data[0,:,:])[0]):

        if(ey!=0 and sx!=0 and ex!=0 and sy!=0):
            print("F",lmi)
            break
        for lmj in range(np.shape(data[0,:,:])[1]):

            if(sx==0):            
                if(np.nanmean(data[0,lmi,:])>0 or np.nanmean(data[0,lmi,:])<0):
                    sx=lmi


            if(sy==0):
                if(np.nanmean(data[0,:,lmj])>0 or np.nanmean(data[0,:,lmj])<0):
                    sy=lmj

            if(ex==0):
                if(np.nanmean(data[0,np.shape(datn[0,:,:])[0]-lmi-1,:])>0 or np.nanmean(data[0,np.shape(data[0,:,:])[0]-lmi-1,:])<0):
                    ex=np.shape(data[0,:,:])[0]-lmi-1

            if(ey==0):
                if(np.nanmean(data[0,:,np.shape(data[0,:,:])[1]-lmj-1])>0 or np.nanmean(data[0,:,np.shape(data[0,:,:])[1]-lmj-1])<0):
                    ey=np.shape(data[0,:,:])[1]-lmj-1

            if(ey!=0 and ex!=0 and sx!=0 and sy!=0):
                break
    print(sx,ex,sy,ey)
    return sx,ex,sy,ey


'''
#Spatial reprojection 

def Reproject_To_Cube(Input_Cube, Target_Cube,i_step=3000,Cube_Name_Save=''):
    
    if os.pathexists("Spectral Cubes/"+Cube_Name_Save):
        print("The cube ", Cube_Name_Save,"Already exists; returning this cube.")
        return SpectralCube.read("Spectral Cubes/" + Cube_Name_Save) 

            
            reheader = copy.deepcopy(sc.hdu.header)
            

            ## Find the number of expected pixels for the new resolution and the location of the left/right, up/down sides 

            #find out which direction the cube is read, left to right or right/to left. (in terms of RA/DEC). Then, do the same for up and down
            
            if sc.header['cdelt1']>0:
                pix_x    = (beam_deg/ovs).to(u.degree).value
                #origin_x = sc.longitude_extrema[0].to(u.degree).value  #, this is the current value, but it doesnt work with the rotation so I use the original values
                origin_x = long_extrema[0].value
                
                if(Force_Origins):
                    origin_x = Force_Value_x[0]#358.6

            else:
                pix_x    = -1.*(beam_deg/ovs).to(u.degree).value
                #origin_x = (sc.longitude_extrema[1]).to(u.degree).value #, this is the current value, but it doesnt work with the rotation so I use the original values:
                origin_x = long_extrema[1].value

                if(Force_Origins):
                    origin_x = Force_Value_x[1]#.9

            if sc.header['cdelt2']>0:
                pix_y    = (beam_deg/ovs).to(u.degree).value
                #origin_y = sc.latitude_extrema[0].to(u.degree).value  #, this is the current value, but it doesnt work with the rotation so I use the original values
                origin_y = lat_extrema[0].value
                if(Force_Origins):
                    origin_y = Force_Value_y[0]#-.6
                    
            else:
                pix_y    = -1.*(beam_deg/ovs).to(u.degree).value
                #origin_y = sc.latitude_extrema[1].to(u.degree).value  #, this is the current value, but it doesnt work with the rotation so I use the original values
                origin_y = lat_extrema[1].value

                if(Force_Origins):
                    origin_y = Force_Value_y[1]#.6
                    
        
            #npix_x   = int(np.ceil(np.diff(sc.longitude_extrema, n=1)[0]/np.abs(pix_x)).value)
            #npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            #npix_x   =int(np.ceil(np.diff([origin_x,origin_x_max])/np.abs(pix_x)))
            #npix_y   =int(np.ceil(np.diff([origin_y,origin_y_max])/np.abs(pix_y)))
            #manually make the cubes overproject because it cuts off data otherwise, because it cannot properly convert the longitude/latitude extrema to ra/dec information:
            npix_x   =int(np.ceil(np.diff([long_extrema.value])/np.abs(pix_x)))
            npix_y   =int(np.ceil(np.diff([lat_extrema.value])/np.abs(pix_y)))
            crpix1 = 0
            crpix2 = 0
            if(Force_Origins):
                
                npix_x   =int(np.diff([Force_Value_x[0]-360,Force_Value_x[1]])/np.abs(pix_x)) 
                npix_y   =int(np.diff(Force_Value_y)/np.abs(pix_y)) 
            
            if(rotation_angleP != 0*u.deg):
                #if there is a rotation, i need additional fov to account for it
                npix_x +=400 #increase the axis size
                npix_y +=200
                crpix1 +=400 # the anchor pixel for the minimum longitude
                crpix2 +=200 # for latitude
            #print(sc.latitude_extrema.to(u.degree).value,origin_y,origin_y_max,npix_y)
            #Correct the header to the expected pixels for the new res

            reheader['cdelt1'] = pix_x
            reheader['cdelt2'] = pix_y

            reheader['naxis1'] = npix_x
            reheader['naxis2'] = npix_y

            reheader['crval1'] = origin_x
            reheader['crval2'] = origin_y

            reheader['crpix1'] = crpix1
            reheader['crpix2'] = crpix2
            
            if(str(sc.wcs).find("GLON")!=-1):
                reheader['CTYPE1'] = "GLON-SIN"
                reheader['CTYPE2'] = "GLAT-SIN"
                
            # remove these things that confuse the reprojection since we won't be using them
            try:
                del reheader['lonpole']
                del reheader['latpole']
                del reheader['wcsaxes']# Dont need these anymore, maybe?
                if(str(sc.wcs).find("GLON")!=-1):

                    del reheader['LBOUND1']
                    del reheader['LBOUND2']
                    del reheader['LBOUND3']
                    del reheader.cards['LBOUND1']
                    del reheader.cards['LBOUND2']
                    del reheader.cards['LBOUND3']

                    reheader['LBOUND1']=0
                    reheader['LBOUND2']=0
                    reheader['LBOUND3']=0
            except Exception as e:
                print("Delete LBOUND if there is one in the header, if not this will pass the error:")
                print(e)
                print("-"*60)
                traceback.print_exc(file=sys.stdout)
                print()

            # regrid cube to target pixel size

            print('start reprojection\n')
            print('check max SC value:',np.nanmax(sc),"SC shape:", np.shape(sc))#These should be a non zero float and the shape of the cube (30,~1000,~1000)
            sc.write("test5.fits",overwrite=True)
            sc.allow_huge_operations=True
            sc2 = sc.reproject(reheader, order='bilinear', use_memmap=True, filled=True) # Had to change reproject.py so it deletes output.np before making a new one
            sc2.write("test6.fits",overwrite=True)

            del sc # save space

            # make a new cube with the reprojcted data (remove all the logs from the old cube)
            
            new = SpectralCube(data=sc2.hdu.data,wcs =WCS(sc2.header),header=sc2.header,mask=sc2.mask)
            new.allow_huge_operations=True
            new = new*sc2[0][0][0].unit
            
            #do this because scs dont like being modified
            del sc2
            sc2 = new
            del new
            
            print('\nend reprojection\n')
            print('check max SC value:',np.nanmax(sc2),"SC shape:", np.shape(sc2))#These should ALSO be a non zero float and the shape of the cube (30,~1000,~1000)

            sc = sc2
            del sc2
            sc.allow_huge_operations=True
            
            if(crop_nans):
                #Mask the pixels outside the fov again after the reprojection to get rid of nan created pixels

                print('fov crop start 2 \n')

                cropped_sc = Crop_Around_Center(sc,rotation_angle,center,FOV,distance)
                del sc
                sc = cropped_sc
                del cropped_sc
                print('fov cropped 2\n')
                # Write the intermediary cubes that will be spliced together
            
            sc.write("Spectral Cubes/"+Cube_Name_Save,overwrite=True)
            
            run_completed=True
            del sc
        except Exception as e:
            print(e)
            print("Failed (unless this says attempt to get argmin of empty sequence)")
            print("-"*60)
            traceback.print_exc(file=sys.stdout)

            
'''
%run ./"0 Imports and functions.ipynb"

/home/ben/miniconda3/bin/python
3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
astropy 5.1.dev153+gb740594dc
spectral_cube 0.6.1.dev22+g003ef16
spectral_cube file path /home/ben/.local/lib/python3.8/site-packages/spectral_cube/__init__.py
reproject 0.8
1.23.1 Numpy
astrodendro_file: /home/ben/.local/lib/python3.8/site-packages/astrodendro/__init__.py
Results will be saved to Directory ./Result Files
Cubes will be saved and loaded with Directory ./Spectral Cubes
Plots will be saved to Directory ./Plots
Dendrograms will be saved to Directory ./Dendrograms


In [3]:
# Pointing_Information

Pointing_Information = {}

#This is the stuff that needs changing between cubes

Pointing_Information["Original_File_Name"] = "12CO_GC_357-358_mosaic.fit"+"_Header_Fix.fits" #the name of the initial SC file.

File_Descriptor='_cmz_CO_J3_2_Splice_'
Pointing_Information["File_Descriptor"] = File_Descriptor
Pointing_Information["target"] = "CMZ"#
Pointing_Information["center"] = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic') #the center of the cmz
Pointing_Information["crop_center"] = SkyCoord('00d00m00.00s  ', '-00d00m00.000s', frame='galactic') #the approximate center of the image, used for cropping

desired_beam_size = 3*u.pc #I add this to the PI later. choose this based on the largest common beam size between the compared observations
distance = 8.178*10**3*u.pc
Pointing_Information["distance"] = distance.to(u.Mpc) #in Mpc
Pointing_Information["target_image_rotation"]=0*u.deg #this is the rotation of the specific image, not the target. (use clockwise rotation angle)
Pointing_Information["target_inclination"]=0*u.deg
Pointing_Information["target_velocity"]=0*u.km/u.s #the speed NGC253 is moving away from us
Pointing_Information["vaxis"]=0 #which axis is the velocity
Pointing_Information["desired_velocity_resolution"] = 2.5*u.km/u.s
ovs = 5 #how much do you desire to oversample the beam by
desired_FOV = [140,700]*u.pc# pc normal to the disk, across the disk
Pointing_Information["Corresponding_Continuum"] = 'gc_850micron_dust.fits' # this is for the band 7, not for HCN J1-0 #The continuum image for this band
#                                                 '4.3pc_beam_CMZ_850um_Cont_140x800pc.fits' # this is the JCMT continuum for 850microns (the CO 3-2 and HCN, HCO 4-3)


print("done")

gc.collect




done


<function gc.collect(generation=2)>

In [4]:

#This stuff will input automagically if the rest is correct

sc = SpectralCube.read("Spectral Cubes/"+Pointing_Information["Original_File_Name"])
header = sc.header
print()
try:
    freq = header["RESTFREQ"]*u.Hz# assuming the header is in Hz
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFREQ"]#
    Pointing_Information['restfreq']=freq         
except:
    freq = header["RESTFRQ"]*u.Hz#
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFRQ"]#            
    Pointing_Information['restfreq']=freq 

######calculate teh beam size of the original image:

if (header['CUNIT1'].find("deg")!=-1):
    CUNIT = 1*u.degree
    Pointing_Information["CUNIT"]=CUNIT
else:
    print("The header should show CUNIT in degrees. If not, just fix this or write CUNIT = the unit it says in the header")
    del jhgasdhgjkahsdkgdfjhsgjgjsdhkfjghjd #this causes an error and stops execution 

    
Pointing_Information["desired_beam_size"] = desired_beam_size #ill put it in a circular beam at this size

Pointing_Information['desired_beam_oversampling'] = ovs
#Pointing_Information['orig_FOV']= Crop_Around_Center(sc,Pointing_Information['target_image_rotation'],Pointing_Information['center'],desired_FOV,Pointing_Information['distance'])[1] #returns the current fov
Pointing_Information['desired_FOV']=desired_FOV

######

Cube_Information = Update_Cube_Information(Pointing_Information,Pointing_Information["Original_File_Name"])






In [6]:
# Take the 3 parts of the cube that have data and splice them together

FD = '_cmz_CO_J3_2_Splice_'
Side="LH_"
Cube_Name_Load_LH = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+FD+Side+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Load_LH =  Cube_Name_Load_LH[0:len(Cube_Name_Load_LH)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'
Side="RH_"
Cube_Name_Load_RH = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+FD+Side+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Load_RH =  Cube_Name_Load_RH[0:len(Cube_Name_Load_RH)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'
Side="FRH_"
Cube_Name_Load_FRH = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+FD+Side+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Load_FRH =  Cube_Name_Load_FRH[0:len(Cube_Name_Load_FRH)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'
#Side="FLH_"
#Cube_Name_Load_FLH = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+FD+Side+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
#Cube_Name_Load_FLH =  Cube_Name_Load_FLH[0:len(Cube_Name_Load_FLH)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

#The save name
FD = "_cmz_CO_J3_2_"
Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+FD+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'


#Load cubes
restfreq = Pointing_Information["restfreq"]
SCR = SpectralCube.read("Spectral Cubes/"+Cube_Name_Load_RH)
SCR = SCR.with_spectral_unit(u.km / u.s, velocity_convention='radio', rest_value=restfreq)
vel,RA,Dec = SCR.world[:,0,0]
#print(vel)
SCL=SpectralCube.read("Spectral Cubes/"+Cube_Name_Load_LH)
SCL = SCL.with_spectral_unit(u.km / u.s, velocity_convention='radio', rest_value=restfreq)
vel,RA,Dec = SCL.world[:,0,0]
#print(vel)
SCFR=SpectralCube.read("Spectral Cubes/"+Cube_Name_Load_FRH)
SCFR = SCFR.with_spectral_unit(u.km / u.s, velocity_convention='radio', rest_value=restfreq)
vel,RA,Dec = SCFR.world[:,0,0]
#print(vel)
#SCFL=SpectralCube.read("Spectral Cubes/"+Cube_Name_Load_FLH)
#SCFL = SCFL.with_spectral_unit(u.km / u.s, velocity_convention='radio', rest_value=restfreq)
#vel,RA,Dec = SCFL.world[:,0,0]


RD = SCR.hdu.data
LD=SCL.hdu.data
FRD = SCFR.hdu.data
#unneeded:
FLD = np.copy(SCFR.hdu.data)*np.nan #SCFL.hdu.data
print(np.shape(RD),len(RD))
print(np.shape(LD))
print(np.shape(FRD))
print(np.shape(FLD))
C = np.copy(LD)
wcs = SCR.wcs
header=SCR.hdu.header
hdu=SCR.hdu
world=SCR.world


for lmi in range(len(RD)):
    for lmj in range(len(RD[lmi])):
        for lmk in range(len(RD[lmi][lmj])):
            if((RD[lmi][lmj][lmk]> 0 or RD[lmi][lmj][lmk]< 0) and (LD[lmi][lmj][lmk]> 0 or LD[lmi][lmj][lmk]< 0)):
                C[lmi][lmj][lmk] = np.nanmean([RD[lmi][lmj][lmk],LD[lmi][lmj][lmk]])
            elif((RD[lmi][lmj][lmk]> 0 or RD[lmi][lmj][lmk]< 0) and (FRD[lmi][lmj][lmk]> 0 or FRD[lmi][lmj][lmk]< 0)):
                C[lmi][lmj][lmk] = np.nanmean([RD[lmi][lmj][lmk],FRD[lmi][lmj][lmk]])
            elif((LD[lmi][lmj][lmk]> 0 or LD[lmi][lmj][lmk]< 0) and (FLD[lmi][lmj][lmk]> 0 or FLD[lmi][lmj][lmk]< 0)):
                C[lmi][lmj][lmk] = np.nanmean([LD[lmi][lmj][lmk],FLD[lmi][lmj][lmk]])
            elif(RD[lmi][lmj][lmk]>0 or RD[lmi][lmj][lmk]<0):
                C[lmi][lmj][lmk] =RD[lmi][lmj][lmk]
            elif(FRD[lmi][lmj][lmk]>0 or FRD[lmi][lmj][lmk]<0):
                C[lmi][lmj][lmk] =FRD[lmi][lmj][lmk]
                #print("Ye",lmi,lmj,lmk)
            elif(LD[lmi][lmj][lmk]>0 or LD[lmi][lmj][lmk]<0):
                C[lmi][lmj][lmk] =LD[lmi][lmj][lmk]
            elif(FLD[lmi][lmj][lmk]>0 or FLD[lmi][lmj][lmk]<0):
                C[lmi][lmj][lmk] =FLD[lmi][lmj][lmk]



Comb = SpectralCube(data=C, wcs=wcs,header=header)
Comb.allow_huge_operations=True
Comb = Comb*SCL[0][0][0].unit



Comb.write(Cube_Name_Save,overwrite=True)
print("done")

(201, 285, 1427) 201
(201, 285, 1427)
(201, 285, 1427)
(201, 285, 1427)


KeyboardInterrupt: 

In [3]:
# Take the 3 parts of the cube that have data and splice them together


Cube_Name_Load_LH = "12CO_GC_001-002_Reprojected.fits"


Cube_Name_Load_RH = "12CO_GC_359-000_Reprojected.fits"


Cube_Name_Load_FRH = "12CO_GC_357-358_Reprojected.fits"



Cube_Name_Save = "12CO_GC_357-003_Reprojected.fits"


SCR = SpectralCube.read("Spectral Cubes/"+Cube_Name_Load_RH)
SCL=SpectralCube.read("Spectral Cubes/"+Cube_Name_Load_LH)

SCFR=SpectralCube.read("Spectral Cubes/"+Cube_Name_Load_FRH)


RD = SCR.hdu.data
LD=SCL.hdu.data
FRD = SCFR.hdu.data
#unneeded:
FLD = np.copy(SCFR.hdu.data)*np.nan #SCFL.hdu.data
print(np.shape(RD),len(RD))
print(np.shape(LD))
print(np.shape(FRD))
print(np.shape(FLD))
C = np.copy(LD)
wcs = SCR.wcs
header=SCR.hdu.header
hdu=SCR.hdu
world=SCR.world


for lmi in range(len(RD)):
    for lmj in range(len(RD[lmi])):
        for lmk in range(len(RD[lmi][lmj])):
            if((RD[lmi][lmj][lmk]> 0 or RD[lmi][lmj][lmk]< 0) and (LD[lmi][lmj][lmk]> 0 or LD[lmi][lmj][lmk]< 0)):
                C[lmi][lmj][lmk] = np.nanmean([RD[lmi][lmj][lmk],LD[lmi][lmj][lmk]])
            elif((RD[lmi][lmj][lmk]> 0 or RD[lmi][lmj][lmk]< 0) and (FRD[lmi][lmj][lmk]> 0 or FRD[lmi][lmj][lmk]< 0)):
                C[lmi][lmj][lmk] = np.nanmean([RD[lmi][lmj][lmk],FRD[lmi][lmj][lmk]])
            elif((LD[lmi][lmj][lmk]> 0 or LD[lmi][lmj][lmk]< 0) and (FLD[lmi][lmj][lmk]> 0 or FLD[lmi][lmj][lmk]< 0)):
                C[lmi][lmj][lmk] = np.nanmean([LD[lmi][lmj][lmk],FLD[lmi][lmj][lmk]])
            elif(RD[lmi][lmj][lmk]>0 or RD[lmi][lmj][lmk]<0):
                C[lmi][lmj][lmk] =RD[lmi][lmj][lmk]
            elif(FRD[lmi][lmj][lmk]>0 or FRD[lmi][lmj][lmk]<0):
                C[lmi][lmj][lmk] =FRD[lmi][lmj][lmk]
                #print("Ye",lmi,lmj,lmk)
            elif(LD[lmi][lmj][lmk]>0 or LD[lmi][lmj][lmk]<0):
                C[lmi][lmj][lmk] =LD[lmi][lmj][lmk]
            elif(FLD[lmi][lmj][lmk]>0 or FLD[lmi][lmj][lmk]<0):
                C[lmi][lmj][lmk] =FLD[lmi][lmj][lmk]



Comb = SpectralCube(data=C, wcs=wcs,header=header)
Comb.allow_huge_operations=True
Comb = Comb*SCL[0][0][0].unit



Comb.write(Cube_Name_Save,overwrite=True)
print("done")

(600, 600, 3480) 600
(600, 599, 3479)
(600, 600, 3480)
(600, 600, 3480)


IndexError: index 3479 is out of bounds for axis 0 with size 3479

In [9]:


# Take the 3 parts of the cube that have data and splice them together

FD = '_cmz_CO_J3_2_Splice_'
Side="LH_"
Cube_Name_Load_LH = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+FD+Side+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Load_LH =  Cube_Name_Load_LH[0:len(Cube_Name_Load_LH)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'
Side="RH_"
Cube_Name_Load_RH = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+FD+Side+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Load_RH =  Cube_Name_Load_RH[0:len(Cube_Name_Load_RH)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'
Side="FRH_"
Cube_Name_Load_FRH = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+FD+Side+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Load_FRH =  Cube_Name_Load_FRH[0:len(Cube_Name_Load_FRH)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'
#Side="FLH_"
#Cube_Name_Load_FLH = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+FD+Side+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
#Cube_Name_Load_FLH =  Cube_Name_Load_FLH[0:len(Cube_Name_Load_FLH)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

#The save name
FD = "_cmz_CO_J3_2_"
Cube_Name_Save = FD + "Spliced_Smoothed_003-357.fits"


#Load cubes
restfreq = Pointing_Information["restfreq"]
SCR = SpectralCube.read("Spectral Cubes/"+Cube_Name_Load_RH)
SCR = SCR.with_spectral_unit(u.km / u.s, velocity_convention='radio', rest_value=restfreq)
vel,RA,Dec = SCR.world[:,0,0]
#print(vel)
SCL=SpectralCube.read("Spectral Cubes/"+Cube_Name_Load_LH)
SCL = SCL.with_spectral_unit(u.km / u.s, velocity_convention='radio', rest_value=restfreq)
vel,RA,Dec = SCL.world[:,0,0]
#print(vel)
SCFR=SpectralCube.read("Spectral Cubes/"+Cube_Name_Load_FRH)
SCFR = SCFR.with_spectral_unit(u.km / u.s, velocity_convention='radio', rest_value=restfreq)
vel,RA,Dec = SCFR.world[:,0,0]
#print(vel)
#SCFL=SpectralCube.read("Spectral Cubes/"+Cube_Name_Load_FLH)
#SCFL = SCFL.with_spectral_unit(u.km / u.s, velocity_convention='radio', rest_value=restfreq)
#vel,RA,Dec = SCFL.world[:,0,0]

reheader = copy.deepcopy(SCFR.hdu.header)

SCRN = SCR.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
SCRN.write("test2.fits",overwrite=True)
del SCR
gc.collect()
SCR = SpectralCube(data=SCRN.hdu.data,wcs =WCS(SCRN.header),header=SCRN.header,mask=SCRN.mask)
SCR.allow_huge_operations=True
SCR = SCR*SCRN[0][0][0].unit




del SCRN
SCLN = SCL.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
del SCL
gc.collect()
SCL = SpectralCube(data=SCLN.hdu.data,wcs =WCS(SCLN.header),header=SCLN.header,mask=SCLN.mask)
SCL.allow_huge_operations=True
SCL = SCL*SCLN[0][0][0].unit

del SCLN

SCFRN = SCFR.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
del SCFR
gc.collect()
SCFR = SpectralCube(data=SCFRN.hdu.data,wcs =WCS(SCFRN.header),header=SCFRN.header,mask=SCFRN.mask)
SCFR.allow_huge_operations=True
SCFR = SCFR*SCFRN[0][0][0].unit
del SCFRN
gc.collect()
#SCFLN = SCFL.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
#del SCFL
#SCFL = SpectralCube(data=SCFLN.hdu.data,wcs =WCS(SCFLN.header),header=SCFLN.header,mask=SCFLN.mask)
#SCFL.allow_huge_operations=True
#SCFL = SCFL*SCFLN[0][0][0].unit

#del SCFLN
SCR.write("test_3.fits",overwrite=True)
SCL.write("test_4.fits",overwrite=True)
SCFR.write("test_5.fits",overwrite=True)


SCR = SpectralCube.read("test_3.fits")
SCL = SpectralCube.read("test_4.fits")
SCFR = SpectralCube.read("test_5.fits")



datn = copy.deepcopy(np.array(SCR.hdu.data))
sx0,ex0,sy0,ey0=Find_Edges(datn)

SCR = SCR[:,sx0:ex0,:]

SCR_Hdu=SCR.hdu
zeros=((SCR_Hdu.data[:,:,:]==0))
bp = np.where(zeros)
SCR_Hdu.data[bp]=np.nan

for lmi in range(len(datn)):
    SCR_Hdu.data[lmi,:,sy0-10:sy0+10]=np.nan
    SCR_Hdu.data[lmi,:,ey0-10:ey0+10]=np.nan#crop a bit around the edges
    #SCR_Hdu.data[lmi,:,ey0-3:ey0+3]=np.nan#crop a bit around the edges
    #SCR_Hdu.data[lmi,:,ey:ey]=np.nan
datn=SCR_Hdu.data
sx,ex,sy,ey=Find_Edges(datn)

SCR = SpectralCube.read(SCR_Hdu)

datn = copy.deepcopy(np.array(SCL.hdu.data))
sx2,ex2,sy2,ey2=Find_Edges(datn)



SCL=SCL[:,sx0:ex0,:]

SCL_Hdu=SCL.hdu
zeros=((SCL_Hdu.data[:,:,:]==0))
bp = np.where(zeros)
SCL_Hdu.data[bp]=np.nan

for lmi in range(len(datn)):
    #SCL_Hdu.data[lmi,:,sy-5:sy+5]=np.nan

    #SCL_Hdu.data[lmi,:,sy2-3:sy2+3]=np.nan
    SCL_Hdu.data[lmi,:,ey2-9:ey2+9]=np.nan

    SCL_Hdu.data[lmi,:,ey2:ey]=np.nan

SCL = SpectralCube.read(SCL_Hdu)


datn = copy.deepcopy(np.array(SCFR.hdu.data))
sx3,ex3,sy3,ey3=Find_Edges(datn)



SCFR=SCFR[:,sx0:ex0,:]

SCFR_Hdu=copy.deepcopy(SCFR.hdu)
zeros=((SCFR_Hdu.data[:,:,:]==0))
bp = np.where(zeros)
SCFR_Hdu.data[bp]=np.nan


for lmi in range(len(datn)):

    SCFR_Hdu.data[lmi,:,sy3-8:sy3+8]=np.nan
    SCFR_Hdu.data[lmi,:,ey3-8:ey3+8]=np.nan


    SCFR_Hdu.data[lmi,:,sy:sy3]=np.nan
    #SCFR_Hdu.data[lmi,:,ey-5:ey+5]=np.nan#Takes care of the cube edges


SCFR = SpectralCube.read(SCFR_Hdu)




SCFL = SpectralCube.read(SCFR_Hdu)


RD = SCR.hdu.data
LD=SCL.hdu.data
FRD = SCFR.hdu.data
FLD = SCFL.hdu.data*np.nan#unused
print("Cube shapes:",np.shape(RD),np.shape(FRD),np.shape(LD),np.shape(FLD))

C = np.copy(LD)
wcs = SCR.wcs
header=SCR.hdu.header
hdu=SCR.hdu
world=SCR.world


for lmi in range(len(RD)):
    for lmj in range(len(RD[lmi])):
        for lmk in range(len(RD[lmi][lmj])):
            if((RD[lmi][lmj][lmk]> 0 or RD[lmi][lmj][lmk]< 0) and (LD[lmi][lmj][lmk]> 0 or LD[lmi][lmj][lmk]< 0)):
                C[lmi][lmj][lmk] = np.mean([RD[lmi][lmj][lmk],LD[lmi][lmj][lmk]])
            elif((RD[lmi][lmj][lmk]> 0 or RD[lmi][lmj][lmk]< 0) and (FRD[lmi][lmj][lmk]> 0 or FRD[lmi][lmj][lmk]< 0)):
                C[lmi][lmj][lmk] = np.mean([RD[lmi][lmj][lmk],FRD[lmi][lmj][lmk]])
            elif((LD[lmi][lmj][lmk]> 0 or LD[lmi][lmj][lmk]< 0) and (FLD[lmi][lmj][lmk]> 0 or FLD[lmi][lmj][lmk]< 0)):
                C[lmi][lmj][lmk] = np.mean([LD[lmi][lmj][lmk],FLD[lmi][lmj][lmk]])
            elif(RD[lmi][lmj][lmk]>0 or RD[lmi][lmj][lmk]<0):
                C[lmi][lmj][lmk] =RD[lmi][lmj][lmk]
            elif(FRD[lmi][lmj][lmk]>0 or FRD[lmi][lmj][lmk]<0):
                C[lmi][lmj][lmk] =FRD[lmi][lmj][lmk]
                #print("Ye",lmi,lmj,lmk)
            elif(LD[lmi][lmj][lmk]>0 or LD[lmi][lmj][lmk]<0):
                C[lmi][lmj][lmk] =LD[lmi][lmj][lmk]
            elif(FLD[lmi][lmj][lmk]>0 or FLD[lmi][lmj][lmk]<0):
                C[lmi][lmj][lmk] =FLD[lmi][lmj][lmk]






    print(np.nanmean(C[lmi]))
    print(lmi)


Comb = SpectralCube(data=C, wcs=wcs,header=header)
Comb.allow_huge_operations=True
Comb = Comb*SCL[0][0][0].unit



Comb.write(Cube_Name_Save,overwrite=True)
print("done")

F 28
26 257 505 998
F 2
1 230 515 987
F 28
25 257 130 524
F 29
25 256 980 1292
Cube shapes: (201, 231, 1427) (201, 231, 1427) (201, 231, 1427) (201, 231, 1427)
-0.010779703935518197
0
-0.01561427316549186
1
-0.009547040288738541
2
-0.012406581961965368
3
-0.008252548692524096
4
-0.008681082226665802
5
-0.003113401250682229
6
0.0019133499402264227
7
0.004217539478204825
8
0.004703554287986799
9
0.0115975762430534
10
0.014689673671389021
11
0.026599155988912937
12
0.04094442211928518
13
0.05610884281443904
14
0.07115523073552683
15
0.07586854300631916
16
0.08309670441065452
17
0.08711792803364364
18
0.08554537693862423
19
0.0834592403501302
20
0.08054213877830632
21
0.0815705368783984
22
0.07943739850431594
23
0.08070133947167557
24
0.08392019409055897
25
0.08463143864314346
26
0.08990984938216834
27
0.08774294421179629
28
0.08923207842973767
29
0.09128601170601876
30
0.09866884543682405
31
0.10313225050562186
32
0.10604238599553566
33
0.10364217281286629
34
0.10746839856195134
35
0.1166